In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow  as tf

2022-06-22 00:22:44.932759: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: ~/hadoop/lib/native
2022-06-22 00:22:44.932792: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import mlflow
tracking_uri = "http://localhost:5000"
mlflow.set_tracking_uri(tracking_uri)

model_uri = "models:/SentimentAnalyzerModel/production"
model = mlflow.sklearn.load_model(model_uri)
model.predict([[4]])

array([[9.]])

In [2]:
data = pd.read_csv('amazon-reviews.csv', encoding="latin-1")

In [3]:
le = LabelEncoder()
data['class'] = le.fit_transform(data.label)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(data.text, data['class'])

In [6]:
maxlen=150
embedding_dim=16
num_words=25000

tokenizer = Tokenizer(num_words=num_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

sequences = tokenizer.texts_to_sequences(X_train)
padded = pad_sequences(sequences, maxlen=maxlen, truncating="post", padding="post")

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, maxlen=maxlen, truncating="post", padding="post")

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(num_words,embedding_dim,input_length=maxlen),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
model.summary()

2022-05-17 23:35:54.197690: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-17 23:35:54.198859: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: ~/hadoop/lib/native
2022-05-17 23:35:54.199130: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: ~/hadoop/lib/native
2022-05-17 23:35:54.199323: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 16)           400000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 32)                544       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 400,577
Trainable params: 400,577
Non-trainable params: 0
_________________________________________________________________


In [8]:
history = model.fit(padded, y_train, epochs=2, validation_data=(test_padded, y_test))

Epoch 1/2
235/235 [==============================] - 3s 7ms/step - loss: 0.6724 - accuracy: 0.6465 - val_loss: 0.6079 - val_accuracy: 0.7480
Epoch 2/2
235/235 [==============================] - 1s 6ms/step - loss: 0.4543 - accuracy: 0.8508 - val_loss: 0.3825 - val_accuracy: 0.8632


In [9]:
pd.DataFrame(history.history).iloc[-1,:]["accuracy"]

0.8507999777793884

In [10]:
test_padded.shape

(2500, 150)

In [1]:
import mlflow

In [2]:
tracking_uri = "http://localhost:5000"
mlflow.set_tracking_uri(tracking_uri)
mlflow.run(".", parameters={"embedding_dim": 16, "epochs": 10, "maxlen": 100, "tracking_uri": tracking_uri})

2022/06/27 17:52:21 INFO mlflow.utils.conda: Conda environment mlflow-1355c61d10b5da360c91c8bcdf1fb26c9b4aaf7b already exists.
2022/06/27 17:52:21 INFO mlflow.projects.utils: === Created directory /tmp/tmpefyfegrh for downloading remote URIs passed to arguments of type 'path' ===
2022/06/27 17:52:21 INFO mlflow.projects.backend.local: === Running command 'source /home/siewe/anaconda3/bin/../etc/profile.d/conda.sh && conda activate mlflow-1355c61d10b5da360c91c8bcdf1fb26c9b4aaf7b 1>&2 && python sentiment.py 16 10 100 http://localhost:5000' in run with ID '794c2e2e97eb4849bfd77791f57cfe1b' === 
2022-06-27 17:52:38.509298: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: ~/hadoop/lib/native
2022-06-27 17:52:38.509320: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           400000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 32)                544       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 400,577
Trainable params: 400,577
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
235/235 [==============================] - 5s 7ms/step - loss: 0.6623 - accuracy: 0.6524 - val_loss: 0.57

2022/06/27 17:54:32 INFO mlflow.projects: === Run (ID '794c2e2e97eb4849bfd77791f57cfe1b') succeeded ===


# Sagemaker Endpoint

In [3]:
import os
import joblib
import mlflow
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def get_predict_params(run_id,dest):
    model_uri = "models:/SentimentAnalyzerModel/production"
    tracking_uri = "http://localhost:5000"

    mlflow.set_tracking_uri(tracking_uri)
    run = mlflow.get_run(run_id)
    mlflow.tracking.MlflowClient().download_artifacts(run.info.run_id,"tokenizer.tok",dest)
    tokenizer = joblib.load(f"{dest}/tokenizer.tok")
    return tokenizer,run.data.params["maxlen"]


def process(sentences,tokenizer, maxlen):
    sequences = tokenizer.texts_to_sequences(sentences)
    padded = pad_sequences(sequences, maxlen=int(maxlen), truncating="post", padding="post")
    return padded

2022-06-27 18:02:08.212607: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: ~/hadoop/lib/native
2022-06-27 18:02:08.212628: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
def predict(data, run_id):
    import boto3
    import json

    endpoint_name = "sentiment-analyzer-api"
    
    # get important run params like tokenizer and maxlen
    dest = f"{os.getcwd()}/"
    tokenizer, maxlen = get_predict_params(run_id, dest)
    
    data = pd.DataFrame(data=process(data, tokenizer, maxlen))

    runtime= boto3.client('runtime.sagemaker')

    # predict on the first row of the dataset
    payload = data.to_json(orient="split")

    runtime_response = runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/json', Body=payload)
    result = json.loads(runtime_response['Body'].read().decode())

    return result

In [8]:
import joblib
run_id="794c2e2e97eb4849bfd77791f57cfe1b"
predict([
    "good to hear that",
    "very disgusting !",
    "such an amazing day"
], run_id)

[{'0': 0.7138702273368835},
 {'0': 0.18724586069583893},
 {'0': 0.774770200252533}]

In [ ]:
model_uri = "models:/SentimentAnalyzerModel/production"
tracking_uri = "http://localhost:5000"
dest = "/home/siewe/Documents/France/Efrei/Cours/M1/Semestre 8/UE - Big Data Fundamentals II - 6/Machine Learning II/Machine Learning Project/downloads"

mlflow.set_tracking_uri(tracking_uri)
#run = mlflow.get_run('52ad95415bf043c09402d2cfa2e1eaa7')

model = mlflow.keras.load_model(model_uri=model_uri)


In [75]:
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository

client = mlflow.tracking.MlflowClient()

name = "SentimentAnalyzerModel"

# passer la version actuellement en staging en Archived
current_staging_version = [int(run.version) for run in client.get_latest_versions(name) if run.current_stage=='Staging'][0]

print("Passing new Model to Production...")
client.transition_model_version_stage(
    name = name,
    version=current_staging_version,
    stage="archived"
)

# passer la version actuellement en production en staging
current_prod_version = [int(run.version) for run in client.get_latest_versions(name) if run.current_stage=='Production'][0]
client.transition_model_version_stage(
    name = name,
    version=current_prod_version,
    stage="staging"
)

print("Passing current Production Model to Staging...")


# recuperer le dernier run et le passer en production
run_id = client.list_run_infos('0')[0].run_id
desc = "A new version of the model automatically created"
runs_uri = "runs:/{}/sentiment-analyzer".format(run_id)
model_src = RunsArtifactRepository.get_underlying_uri(runs_uri)
client.create_model_version(name, model_src, run_id, description=desc)

new_version = [int(run.version) for run in client.get_latest_versions(name) if run.current_stage=='None'][0]

print("Passing new Model to Production...")
client.transition_model_version_stage(
    name = name,
    version=new_version,
    stage="production"
)


print('done')

Passing new Model to Production...
Passing current Production Model to Staging...


2022/05/29 16:26:26 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: SentimentAnalyzerModel, version 16


Passing new Model to Production...
done


In [74]:
client.get_latest_versions(name)

[<ModelVersion: creation_timestamp=1653834243187, current_stage='Archived', description='A new version of the model automatically created', last_updated_timestamp=1653834299477, name='SentimentAnalyzerModel', run_id='4b9fdf5ca3564ccfaf6bdefbb356ceed', run_link='', source='s3://mlflow-bucket-1606/0/4b9fdf5ca3564ccfaf6bdefbb356ceed/artifacts/sentiment-analyzer', status='READY', status_message='', tags={}, user_id='', version='15'>,
 <ModelVersion: creation_timestamp=1653832418842, current_stage='None', description='A new version of the model automatically created', last_updated_timestamp=1653832418842, name='SentimentAnalyzerModel', run_id='4b9fdf5ca3564ccfaf6bdefbb356ceed', run_link='', source='s3://mlflow-bucket-1606/0/4b9fdf5ca3564ccfaf6bdefbb356ceed/artifacts/sentiment-analyzer', status='READY', status_message='', tags={}, user_id='', version='9'>,
 <ModelVersion: creation_timestamp=1653833756822, current_stage='Production', description='A new version of the model automatically creat

In [18]:
import joblib
tokenizer = joblib.load(f"{dest}/tokenizer.tok")
predict(["good to hear that","really disgusting! how can you be so stupid?","such an amazing day"], tokenizer, maxlen)

[{'0': 0.5535424947738647},
 {'0': 0.46320345997810364},
 {'0': 0.5110406875610352}]